In [662]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This notebook creates a figure showing the north and
# northwest arcs of the Sunburst Arc in LyC and Lyman-alpha

In [663]:
import os

import numpy as np

from reproject import reproject_interp

from regions import CircleSkyRegion, RectangleSkyRegion

from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.visualization.wcsaxes import add_scalebar

import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
import matplotlib.patheffects as pe
import matplotlib.patches as patches

In [664]:
def plot():

    def rotate_wcs(wcs, angle):

        '''
        Rotates a WCS projection by some angle

        Parameters:

            wcs : astropy.wcs.wcs.WCS
                The original WCS projection

            angle : int
                The angle (in degrees) to rotate the WCS projection

        Returns:

            rotated_wcs : astropy.wcs.wcs.WCS
                The rotated WCS projection
        '''

        # Define the rotation angle
        angle = Angle(angle, unit='deg')

        # Get the CD matrix from the original WCS projection
        cd_matrix = wcs.wcs.cd

        # Calculate the rotation matrix
        cos_angle = np.cos(angle.rad)
        sin_angle = np.sin(angle.rad)
        rotation_matrix = np.array([[cos_angle, -sin_angle], [sin_angle, cos_angle]], dtype=np.float64)

        # Apply the rotation to the CD matrix
        rotated_cd_matrix = np.dot(cd_matrix, rotation_matrix)

        # Update the CD matrix in the rotated WCS projection
        rotated_wcs = wcs.deepcopy()
        rotated_wcs.wcs.cd = rotated_cd_matrix

        return rotated_wcs

    # Dictionary containing information of the images of the different source plane regions of the Sunburst Arc appearing in the zoom-in inset axes, including their coordinates 
    # (RA, declination), color, and label orientation. The image labels and colors follow the terminology and color scheme of Sharon et al. (2022) (ApJ, 941, 203)
    images = {
        '1.1': [237.530818, -78.182515, '#ff0000', 'left', 45], '1.2': [237.525561, -78.182754, '#ff0000', 'right', 60], '1.3': [237.524859, -78.182817, '#ff0000', 'right', 75],
        '1.4': [237.519097, -78.183181, '#ff0000', 'right', -45], '1.5': [237.518255, -78.183256, '#ff0000', 'right', 45], '1.6': [237.517455, -78.183354, '#ff0000', 'right', -45],
        '1.7': [237.509198, -78.184647, '#ff0000', 'left', -45], '1.8': [237.501508, -78.186259, '#ff0000', 'right', 45], '1.9': [237.499785, -78.186745, '#ff0000', 'right', 45],
        '1.10': [237.498917, -78.187077, '#ff0000', 'right', 45],

        '2.1': [237.528078, -78.182604, '#ff6f00', 'left', 60], '2.2': [237.527379, -78.182635, '#ff6f00', 'left', 60], '2.3': [237.523906, -78.182902, '#ff6f00', 'left', 45],
        '2.4': [237.520126, -78.183100, '#ff6f00', 'right', -45], '2.7': [237.508784, -78.184748, '#ff6f00', 'left', -45], '2.8': [237.503795, -78.185731, '#ff6f00', 'left', 45],

        '3.3': [237.523472, -78.182926, '#00eb00', 'right', 75], '3.4': [237.520701, -78.183058, '#00eb00', 'right', 75], '3.7': [237.508271, -78.184839, '#00eb00', 'right', 60],
        '3.8': [237.504896, -78.185501, '#00eb00', 'right', 60],

        '4.3': [237.523614, -78.182962, '#9C27B0', 'right', -45], '4.4': [237.520679, -78.183110, '#9C27B0', 'left', 45], '4.7': [237.509090, -78.184749, '#9C27B0', 'left', 30],
        '4.8': [237.5025603 - 0.0000547, -78.1861470 + 0.0000541, '#9C27B0', 'right', 75], '4.9': [237.499970, -78.186785, '#9C27B0', 'left', 45], '4.10': [237.499155, -78.187089, '#9C27B0', 'left', 45],

        '5.1': [237.530472, -78.182525, '#c62828', 'right', -45], '5.2': [237.525924, -78.182728, '#c62828', 'left', 60], '5.3': [237.524665, -78.182835, '#c62828', 'left', 60],
        '5.4': [237.519300, -78.183166, '#c62828', 'right', 75],

        '6.1': [237.530282, -78.182473, '#e53935', 'left', -45], '6.2': [237.526374, -78.182639, '#e53935', 'right', 45],

        '7.1': [237.529282, -78.182482, '#ef5350', 'right', 45], '7.2': [237.527034, -78.182569, '#ef5350', 'right', 45], '7.3': [237.523693, -78.182846, '#ef5350', 'left', -60],
        '7.4': [237.520049, -78.183013, '#ef5350', 'right', 45], '7.7': [237.507458, -78.184894, '#ef5350', 'right', 45], '7.8': [237.505953, -78.185193, '#ef5350', 'right', 45],

        '8.1': [237.528607, -78.182557, '#ef9a9a', 'right', 45], '8.2': [237.527136, -78.182623, '#ef9a9a', 'right', -45], '8.3': [237.523917, -78.182875, '#ef9a9a', 'left', -45],
        '8.4': [237.520032, -78.183080, '#ef9a9a', 'right', 30], '8.7': [237.508469, -78.184775, '#ef9a9a', 'left', -45], '8.8': [237.504388, -78.185573, '#ef9a9a', 'right', 60],

        '9.3': [237.523676, -78.182928, '#43a047', 'left', 60], '9.4': [237.520492, -78.183090, '#43a047', 'right', -45], '9.8': [237.504180, -78.185652, '#43a047', 'left', 45],

        '10.3': [237.523251, -78.182942, '#8bc34a', 'right', -45], '10.4': [237.521002, -78.183046, '#8bc34a', 'left', 45], '10.7': [237.508163, -78.184876, '#8bc34a', 'left', 60],
        '10.8': [237.505064, -78.185473, '#8bc34a', 'left', 45],

        '11.3': [237.523067, -78.182913, '#66bb6a', 'right', 45], '11.4': [237.521062, -78.182998, '#66bb6a', 'left', -45],

        '12.3': [237.522753, -78.182948, '#aed581', 'right', -45], '12.4': [237.521510, -78.182999, '#aed581', 'left', -45],

        '13.7': [237.507614, -78.185015, '#a5d6a7', 'right', 60], '13.8': [237.506255, -78.185277, '#a5d6a7', 'right', 45],

        '14.7': [237.507415, -78.185061, '#dcedc8', 'left', 75], '14.8': [237.506400, -78.185256, '#dcedc8', 'right', -60],

        '15.7': [237.507203, -78.185102, '#1b5e20', 'right', 45], '15.8': [237.506595, -78.185218, '#1b5e20', 'left', 45],

        '101.1': [237.532701, -78.182363, '#FF00FF', 'right', 0], '101.6': [237.517073, -78.183258, '#FF00FF', 'right', 0], '101.8': [237.503649, -78.185600, '#FF00FF', 'right', 15]
        }

    # Dictionary with information describing how to place label text 
    # around images based on a keyword
    image_label_positions = {
        'left': [-5, -5, 'right', 'center'],
        'right': [5, 5, 'left', 'center']
        }

    # Dictionary containing information about the position, orientation,
    # and style of the slit apertures
    slits = {
        'M5' : ['15:50:01.1649 -78:11:07.822', 0.85, 313, 'dashed'],
        'M4' : ['15:50:04.9279 -78:10:59.032', 0.85, 0, 'dashed'],
        'M6' : ['15:50:06.6389 -78:10:57.412', 0.85, 12, 'dashed'],
        'M3' : ['15:50:00.6009 -78:11:09.912', 0.85, 298, 'dashed'],
        'M0' : ['15:50:04.4759 -78:10:59.652', 1.0, 349, 'solid'],
        'M2' : ['15:49:59.7480 -78:11:13.482', 0.85, 52, 'solid'],
        'M7' : ['15:50:07.3959 -78:10:56.962', 0.85, 0, 'solid'],
        'M8' : ['15:49:59.9499 -78:11:12.242', 0.85, 330, 'solid'],
        'M9' : ['15:50:00.3719 -78:11:10.512', 0.85, 302, 'solid']
    }

    # Positions of the slit labels in terms of fractions of axes
    slit_label_positions = {
        'M5' : (0.305,0.1),
        'M4' : (0.65,0.1),
        'M6' : (0.41,0.1),
        'M3' : (0.47,0.1),
        'M0' : (0.87,0.1),
        'M2' : (0.86,0.7),
        'M7' : (0.2,0.1),
        'M8' : (0.87,0.05),
        'M9' : (0.685,0.77)
    }

    # Structured list of the slit apertures (by their slit ID) that appear on 
    # each arc. The first sublist is the apertures on the north arc and the 
    # second is the apertures on the northwest arc
    slits_arc = [['M7', 'M6', 'M4', 'M0'], ['M5', 'M3', 'M9', 'M8', 'M2']]
    
    # The labels specifying the displayed arc and what the data shows
    arc_labels = np.array(['N', 'NW'], dtype=str)
    filter_labels = np.array(['Lyα (F555W)', 'Lyα (F390W)', 'F275W'], dtype=str)

    # Establish common directories
    home = os.getcwd()
    data = f'{home}/data'
    figs = f'{home}/figs'
    results = f'{home}/results'

    # Establish file paths to the necessary files
    hst_f275w_file = f'{data}/hst/V5.0_PSZ1G311.65-18.48_F275W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits'
    hst_f606w_file = f'{data}/hst/V5.0_PSZ1G311.65-18.48_F606W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits'
    lya_nb_f390w_file = f'{results}/Lya_cont_sub_F390W.fits'
    lya_nb_f555w_file = f'{results}/Lya_cont_sub_F555W.fits'

    # Put the files we will display data from into an array. We will loop over this array
    # to make the figure
    files = np.array([lya_nb_f555w_file, lya_nb_f390w_file, hst_f275w_file], dtype=str)

    # Position, dimensions, and orientations of the boxes defining the areas of the inset, zoom-in axes, 
    # defined in pixel coordinates of the HST F606W observation's WCS projection
    width, height = 464, 86
    centers = np.array([[4591,5303],[5093,4944]], dtype=int)
    angles = np.array([-19, -48], dtype=np.float64)
    compass_positions = np.array([[464-2,12],[464-2,20]], dtype=int)

    # Get the WCS projection of the HST F606W observations
    hst_f606w_wcs = WCS(fits.getheader(hst_f606w_file))

    # Loop to create the WCS projections of each set of panels

    # Create an empty list to append the rotated WCS projections to
    WCSs = []

    # For each angle that each WCS projection 
    # is rotated relative to the original WCS projection
    for i, angle in enumerate(angles):

        # Rotate the original WCS by the angle
        wcs_rotated = rotate_wcs(hst_f606w_wcs, angle)

        # Add the rotated WCS to the list of rotated WCS projections
        WCSs = np.append(WCSs, wcs_rotated)

    # Create the figure and axes
    fig, ax = plt.subplots(6,1, figsize=(10,10*516/464))

    # For the file of each image we want to plot
    for i, file in enumerate(files):

        # Get the image data from the file
        data = fits.getdata(file)

        # For each set of axes we will plot portions of the image on
        for j, axes in enumerate([ax[0 + i], ax[3 + i]]):

            # Rotate the data so that the pixel grid aligns with the axes
            data_rotated, _ = reproject_interp((data, hst_f606w_wcs), WCSs[j], (6000,6000))

            # Instantiate the axes with the appropriate rotated WCS
            axes = plt.subplot(611 + 3 * j + i, projection=WCSs[j])

            # Set the minimum and maximum values used to create the colormap in the image.
            # The F275W has a much different absolute scale, so we assign a different 
            # minimum and maximum
            vmin = -0.01 if i == 2 else -0.1
            vmax = 0.01 if i == 2 else 0.1

            # Plot the data on the axes with a symmetric, logarithmic color scale
            axes.imshow(data_rotated, origin='lower', cmap='coolwarm', norm=SymLogNorm(vmin=vmin, vmax=vmax, linthresh=0.0001, linscale=0.0001, base=10), 
                interpolation='none')

            # Get the pixel coordinate center of the panel in the new, rotated WCS projection
            center = WCSs[j].world_to_pixel_values([hst_f606w_wcs.pixel_to_world_values([[*centers[j]]])[0]])[0]

            # Set the axes limits to match the footprint of the arc zoom-in insets in the overview map figure
            axes.set_xlim(center[0] - width/2, center[0] + width/2)
            axes.set_ylim(center[1] - height/2, center[1] + height/2)

            # For each coordinate axis
            for k, coord_ax in enumerate([*axes.coords]):

                # Disable the ticks, tick labels, and axis label of the cordinate axis
                coord_ax.set_ticks_visible(False)
                coord_ax.set_ticklabel_visible(False)
                coord_ax.set_axislabel('')

            # Add a label to the top left corner of the axes indicating which arc the panel shows
            arc_label = axes.annotate(arc_labels[j], xy=(0.01, 1 - 0.01 * 464 / 86), xycoords='axes fraction', ha='left', va='top', fontsize=10, fontweight='bold')

            # Add a label to the top right corner of the axes describing the displayed data
            filter_label = axes.annotate(filter_labels[i], xy=(0.99, 1 - 0.01 * 464 / 86), xycoords='axes fraction', ha='right', va='top', fontsize=10, fontweight='bold')

            # Add an outline around the labels to make them easier to read
            arc_label.set_path_effects([pe.withStroke(linewidth=1, foreground='white')])
            filter_label.set_path_effects([pe.withStroke(linewidth=1, foreground='white')])

            # Add a compass to the axes

            # Add the arrows
            n_arrow = axes.annotate('', xy=(compass_positions[j][0] + 20 * np.cos(np.radians(90 - angles[j])), compass_positions[j][1] + 20 * np.sin(np.radians(90 - angles[j]))), 
                xytext=(compass_positions[j][0], compass_positions[j][1]), ha='right', va='bottom', color='black', 
                arrowprops={'facecolor':'black', 'edgecolor':'black', 'arrowstyle':'-|>'}, xycoords='axes pixels')
            e_arrow = axes.annotate('', xy=(compass_positions[j][0] + 20 * np.cos(np.radians(180 - angles[j])), compass_positions[j][1] + 20 * np.sin(np.radians(180 - angles[j]))), 
                xytext=(compass_positions[j][0], compass_positions[j][1]), ha='right', va='top', color='black', 
                arrowprops={'facecolor':'black', 'edgecolor':'black', 'arrowstyle':'-|>'}, xycoords='axes pixels')

            # The arrows (for some reason) do not completely reach to the specified origin, so add a circle at their specified origin to conceal this
            axes.add_patch(patches.Circle((center[0] - 464/2 + compass_positions[j][0] - 2, center[1] - 86/2 + compass_positions[j][1]), radius=2, facecolor='black', transform=axes.transData))

            # Add text labels indicating the cardinal directions (in a (RA, dec) coordinate system) of each arrow
            n_label = axes.annotate('N', xy=(compass_positions[j][0] + 18 * np.cos(np.radians(90 - angles[j])), compass_positions[j][1] + 18 * np.sin(np.radians(90 - angles[j]))), 
                rotation=-angles[j], rotation_mode='anchor', xycoords='axes pixels', color='black', ha='center', va='bottom', fontsize='x-small', fontweight='bold')
            e_label = axes.annotate('E', xy=(compass_positions[j][0] + 18 * np.cos(np.radians(180 - angles[j])), compass_positions[j][1] + 18 * np.sin(np.radians(180 - angles[j]))), 
                rotation=-angles[j], rotation_mode='anchor', xycoords='axes pixels', color='black', ha='right', va='center', fontsize='x-small', fontweight='bold')

            # Add an outline around the labels to make them easier to read
            n_arrow.set_path_effects([pe.withStroke(linewidth=1, foreground='white')])
            e_arrow.set_path_effects([pe.withStroke(linewidth=1, foreground='white')])
            n_label.set_path_effects([pe.withStroke(linewidth=1, foreground='white')])
            e_label.set_path_effects([pe.withStroke(linewidth=1, foreground='white')])

            # Add a scale bar to the axes
            add_scalebar(axes, 1 * u.arcsecond, label='1\'\'', corner='bottom left')#, fontsize=4)

            # For each slit aperture ID
            for k, slit_id in enumerate(slits_arc[j]):

                # Instantiate the slit as a RectangleSkyRegion
                slit_region = RectangleSkyRegion(center=SkyCoord(slits[slit_id][0], unit=(u.hourangle, u.deg), frame='fk5'), 
                    width=slits[slit_id][1] * u.arcsecond, height=10 * u.arcsecond, angle=slits[slit_id][2] * u.deg)

                # Convert the slit region object to a pixel-based representation
                slit_region_pxl = slit_region.to_pixel(WCSs[j])
    
                # Plot the slit region on the axes
                slit_region_pxl.plot(ax=axes, color='#D41159', lw=1, linestyle=slits[slit_id][3])

                # Add a label to the slit
                label = axes.annotate(slit_id, xy=slit_label_positions[slit_id], color='#D41159', fontsize=10, fontweight='bold',
                    xycoords='axes fraction')

                # Add an outline around the labels to make them easier to read
                label.set_path_effects([pe.withStroke(linewidth=1, foreground='white')])

            # For each image of a source plane region that appears in one of the panels of the two arc segments
            for k, image in enumerate(images):

                # Get the color used for the region in the lens model from Sharon+2022 (ApJ, 941, 203)
                color = images[image][2]

                # Get the position keyword used to orient the label text
                loc = images[image][3]

                # Get the angle to orient the text around the image
                angle = images[image][4]

                # The radius (half width) of the circle (square) around an image. Image 4.8 requires a larger radius
                # because it is a series of images of unclear, but similar relation
                radius = 0.4 * u.arcsecond if image == '4.8' else 0.1 * u.arcsecond

                # Pixel coordinates of a manual offset for all label text. This is necessary because the astrometry 
                # used for the presented image coordinates in Sharon et al. (2022) (ApJ, 941, 203) appears to be less 
                # accurate than available with the current data reduction
                offset = [1.1, 1.0]

                # Instantiate a circular region (unless the image is one of the images of the LyC-leaking region; then instantiate it as a rectangular (square) region). 
                # The manual coordinate offset from the listed dictionary value is necessary because of the astrometry difference mentioned above
                image_region = RectangleSkyRegion(center=SkyCoord(images[image][0] + 0.0000547, images[image][1] - 0.0000541, unit='deg', frame='fk5'), width=2 * radius, height=2 * radius, angle=angles[j] * u.deg) if image[:2] == '1.' \
                    else CircleSkyRegion(center=SkyCoord(images[image][0] + 0.0000547, images[image][1] - 0.0000541, unit='deg', frame='fk5'), radius=radius)

                # Convert the slit region object to a pixel-based representation
                image_region_pxl = image_region.to_pixel(WCSs[j])
    
                # Plot the slit region on the axes
                image_region_pxl.plot(ax=axes, color=color, lw=1.0)

                # Get the pixel coordinates of the image using the same manual coordinate offset mentioned previously
                image_pxl_coords = WCSs[j].world_to_pixel_values([[images[image][0] + 0.0000547, images[image][1] - 0.0000541]])
                image_pxl_coords = np.array([image_pxl_coords[0][0], image_pxl_coords[0][1]])# + np.array([dict[loc][0], dict[loc][1]], dtype=float)# - np.array([points[j][1], points[j][0]], dtype=float)

                # The radius of the text label in pixel coordinates from the coordinate center of the image. Image 4.8 requires a much larger
                # radius because the circle around it is much larger
                radius_text = [16,16] if image == '4.8' else image_label_positions[loc]

                # Add additional padding to the text for images of the LyC-leaking region since it is marked with a square (the corners jut out 
                # farther than a circle would)
                radius_text = [radius_text[0], radius_text[1]] + np.sign([radius_text[0], radius_text[1]]) if image[:2] == '1.' else radius_text

                # Calculate the position of the annotation in pixel coordinates
                xytext = (image_pxl_coords[0] + radius_text[0] * np.cos(np.radians(angle)),
                    image_pxl_coords[1] - offset[j] + radius_text[1] * np.sin(np.radians(angle)))

                # Annotate the image with a label
                label = axes.annotate(image.upper(), xy=xytext, rotation=angle, rotation_mode='anchor', 
                    ha=image_label_positions[loc][2], va=image_label_positions[loc][3], color=color, fontsize='small')

                # Add an outline around the labels to make them easier to read
                label.set_path_effects([pe.withStroke(linewidth=1, foreground='black')])

    # Save the figure
    fig.savefig(f'{figs}/lya_and_lyc_maps.pdf', bbox_inches='tight')

In [ ]:
plot()